In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

In [2]:
rawfeats = pd.read_csv('data/dengue_features_train.csv')
rawlabels = pd.read_csv('data/dengue_labels_train.csv')
rawfeats['total_cases'] = rawlabels['total_cases']

# San Juan

In [3]:
sj = rawfeats[rawfeats.city=='sj']
print(sj.shape)

(936, 25)


In [4]:
def process_SJ(rawfeats):
    #Returns a dataset ready for splitting/training or prediction
    #Fill nas with interpolation
    feats = rawfeats.interpolate(method='linear')
    #Replace week 53 with week 52
    feats.loc[:,'weekofyear'] = np.where(feats.weekofyear > 52, 52, feats.weekofyear)
    #Scale then average temperature readings
    tempscols_to_average = feats.loc[:,['reanalysis_max_air_temp_k', 'station_avg_temp_c',
       'reanalysis_avg_temp_k', 'reanalysis_min_air_temp_k',
       'station_min_temp_c', 'reanalysis_dew_point_temp_k',
       'reanalysis_air_temp_k']]
    scaled_temps = pd.DataFrame(MinMaxScaler().fit_transform(tempscols_to_average), 
                            columns=tempscols_to_average.columns)
    feats.loc[:,'temps_mean'] = scaled_temps.mean(axis=1)
    #Boolean season variables
    cutoffs = [11, 26, 42]
    feats['winter'] = np.where((feats.weekofyear<cutoffs[0]), 1, 0)

    feats['spring'] = np.where((feats.weekofyear>=cutoffs[0]) &
                               (feats.weekofyear<cutoffs[1]), 1, 0)
    feats['summer'] = np.where((feats.weekofyear>=cutoffs[1]) &
                               (feats.weekofyear<cutoffs[2]), 1, 0)
    feats['fall'] = np.where((feats.weekofyear>=cutoffs[2]), 1, 0)
    #drop unneeded columns
    keep = ['total_cases','spring', 'summer', 'fall', 'station_max_temp_c',
       'temps_mean', 'reanalysis_relative_humidity_percent',
       'reanalysis_specific_humidity_g_per_kg','reanalysis_precip_amt_kg_per_m2']
    for col in feats.columns:
        if col not in keep:
            feats = feats.drop(col, axis=1)
    
    #add shifted feats, 3 weeks
    to_shift = ['station_max_temp_c', 'temps_mean','reanalysis_relative_humidity_percent',
       'reanalysis_specific_humidity_g_per_kg','reanalysis_precip_amt_kg_per_m2']
    
    for i in to_shift:
        feats[i+'_1lag'] = feats[i].shift(-1)
        feats[i+'_2lag'] = feats[i].shift(-2)
        feats[i+'_3lag'] = feats[i].shift(-3)
    feats = feats.fillna(method='ffill')
    return feats


In [5]:
sj = process_SJ(sj)
sj.shape

(936, 24)

In [6]:
#Split
sj_X = sj.drop(['total_cases'], axis=1)
sj_y = sj.total_cases

X_train_sj, X_test_sj, y_train_sj, y_test_sj = train_test_split(
    sj_X, sj_y, test_size=0.3)
print(X_train_sj.shape)
print(X_test_sj.shape)

(655, 23)
(281, 23)


# Iquitos 

In [7]:
iq = rawfeats[rawfeats.city=='iq']
print(iq.shape)

(520, 25)


In [8]:
def process_IQ(rawfeats):
    #Returns a dataset ready for splitting/training or prediction
    #Fill nas with interpolation
    feats = rawfeats.interpolate(method='linear')
    #Replace week 53 with week 52
    feats.loc[:,'weekofyear'] = np.where(feats.weekofyear > 52, 52, feats.weekofyear)
    #season features
    cutoffs = [15, 31, 46]
    feats['fall'] = np.where((feats.weekofyear<cutoffs[0]), 1, 0)

    feats['winter'] = np.where((feats.weekofyear>=cutoffs[0]) &
                               (feats.weekofyear<cutoffs[1]), 1, 0)
    feats['spring'] = np.where((feats.weekofyear>=cutoffs[1]) &
                               (feats.weekofyear<cutoffs[2]), 1, 0)
    feats['summer'] = np.where((feats.weekofyear>=cutoffs[2]), 1, 0)
    #drop unneeded columns
    keep = ['total_cases',
       'spring', 'summer', 'fall', 'station_avg_temp_c',
       'reanalysis_min_air_temp_k','station_min_temp_c',
       'reanalysis_dew_point_temp_k','reanalysis_tdtr_k',
       'reanalysis_specific_humidity_g_per_kg',
       'precipitation_amt_mm']
    
    for col in feats.columns:
        if col not in keep:
            feats = feats.drop(col, axis=1)
    
    #add shifted feats, 3 weeks
    to_shift = ['station_avg_temp_c','reanalysis_min_air_temp_k', 'station_min_temp_c',
       'reanalysis_dew_point_temp_k', 'reanalysis_tdtr_k', 
       'reanalysis_specific_humidity_g_per_kg','precipitation_amt_mm']
    
    for i in to_shift:
        feats[i+'_1lag'] = feats[i].shift(-1)
        feats[i+'_2lag'] = feats[i].shift(-2)
        feats[i+'_3lag'] = feats[i].shift(-3)
    feats = feats.fillna(method='ffill')
    return feats

In [9]:
iq = process_IQ(iq)
iq.shape

(520, 32)

In [10]:
iq_X = iq.drop(['total_cases'], axis=1)
#X2 = iqshiftedfeats.drop(['total_cases', 'city', 'year', 'weekofyear'], axis=1) #for regularizing techniques
iq_y = iq.total_cases

In [11]:
X_train_iq, X_test_iq, y_train_iq, y_test_iq = train_test_split(
    iq_X, iq_y, test_size=0.3)
print(X_train_iq.shape)
print(X_test_iq.shape)

(364, 31)
(156, 31)


### Grid Searched Random Forest Submission

In [12]:
#Define the parameters we want to cycle through
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10,20,30,40,50],
    'max_features': [2, 5, 'auto'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [100, 300, 500]
}

In [13]:
# rfr = RandomForestRegressor()
# gs = GridSearchCV(estimator=rfr, param_grid=param_grid,cv=3, n_jobs=-1)

# #Fit the grid search to data for SJ
# gs.fit(X_train_sj, y_train_sj)

# #Let's see what came out best:
# gs.best_params_

In [12]:
sj_rf_params = {'bootstrap': True,
 'max_depth': 40,
 'max_features': 5,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 100}

In [15]:
# rfr = RandomForestRegressor()
# gs = GridSearchCV(estimator=rfr, param_grid=param_grid,cv=3, n_jobs=-1)

# #Fit the grid search to data for SJ
# gs.fit(X_train_iq, y_train_iq)

# #Let's see what came out best:
# gs.best_params_

In [13]:
iq_rf_params = {'bootstrap': True,
 'max_depth': 20,
 'max_features': 2,
 'min_samples_leaf': 3,
 'min_samples_split': 4,
 'n_estimators': 100}

In [14]:
sj_rfr = RandomForestRegressor(**sj_rf_params)
iq_rfr = RandomForestRegressor(**iq_rf_params)

In [15]:
sj_rfr.fit(sj_X, sj_y)
iq_rfr.fit(iq_X, iq_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

### Test Data processing

In [16]:
testdata = pd.read_csv('data/dengue_features_test.csv')

In [17]:
sj_test = testdata[testdata.city=='sj'].copy()
sj_test = process_SJ(sj_test)
#23 feat columns
sj_test.shape

(260, 23)

In [18]:
iq_test = testdata[testdata.city=='iq'].copy()
iq_test = process_IQ(iq_test)
#31 feat columns
iq_test.shape

(156, 31)

In [19]:
sj_pred = sj_rfr.predict(sj_test).astype(int)
iq_pred = iq_rfr.predict(iq_test).astype(int)

In [20]:
#zero out negatives if applicable
print(np.min(sj_pred))
print(np.min(iq_pred))


9
2


In [24]:
submission = pd.read_csv('data/submission_format.csv',
                            index_col=[0, 1, 2])

In [25]:
submission.total_cases = np.concatenate([sj_pred, iq_pred])
submission.to_csv("rfr_with_3wk_lag.csv")
#This scored a 24.9, a point better than the benchmark

------

In [22]:
iq_pred

array([ 5,  6,  4,  2,  2,  2,  2,  4,  4,  5,  3,  4,  3,  5,  6, 10, 10,
       10, 10, 11, 14,  8,  9,  9,  8,  6,  7,  7,  5,  5,  5,  8, 13, 12,
        9, 10, 11, 12, 11, 12,  8, 11,  8,  8,  5,  5,  5,  4,  3,  3,  2,
        2,  2,  2,  2,  2,  2,  2,  3,  3,  4,  3,  4,  5,  5,  6,  8,  8,
        9,  9, 11, 12, 10, 11, 11, 10, 13, 12,  9,  7,  7,  8,  9,  8,  8,
        7,  7,  5,  7,  6,  9, 11, 10,  9,  7,  7,  6,  5,  4,  3,  4,  3,
        2,  3,  3,  3,  3,  3,  2,  3,  2,  2,  2,  2,  2,  2,  3,  5,  7,
        7, 10, 10, 13, 14, 11, 12, 11, 12,  8,  7,  9,  8, 13,  9, 12,  8,
       13, 13, 16, 16, 13, 12,  8, 11,  7,  8,  9,  8,  7,  6,  4,  4,  3,
        3,  3,  4])

## Negative Binomial Regression

In [149]:
alpha_params = 10 ** np.arange(-8, -2, dtype=np.float64)

#### SJ

In [150]:
#create statsmodels formula
formula = 'total_cases ~ '
for i in sj.columns:
    if i != 'total_cases':
        formula = formula + str(i) + ' + '
formula =  formula[:-3] #trim the last plus sign
formula

'total_cases ~ reanalysis_precip_amt_kg_per_m2 + reanalysis_relative_humidity_percent + reanalysis_specific_humidity_g_per_kg + station_max_temp_c + temps_mean + spring + summer + fall + station_max_temp_c_1lag + station_max_temp_c_2lag + station_max_temp_c_3lag + temps_mean_1lag + temps_mean_2lag + temps_mean_3lag + reanalysis_relative_humidity_percent_1lag + reanalysis_relative_humidity_percent_2lag + reanalysis_relative_humidity_percent_3lag + reanalysis_specific_humidity_g_per_kg_1lag + reanalysis_specific_humidity_g_per_kg_2lag + reanalysis_specific_humidity_g_per_kg_3lag + reanalysis_precip_amt_kg_per_m2_1lag + reanalysis_precip_amt_kg_per_m2_2lag + reanalysis_precip_amt_kg_per_m2_3lag'

In [157]:
sjtrain, sjtest, _, _ = train_test_split(
    sj, [0 for _ in range(sj.shape[0])], test_size=0.3)
for alpha in grid:
    model = smf.glm(formula=formula, data=sjtrain,
                    family=sm.families.NegativeBinomial(alpha=alpha))
    results = model.fit()
    predictions = results.predict(sjtest).astype(int)
    score = eval_measures.meanabs(predictions, sjtest.total_cases)
    best_score = 1000
    if score < best_score:
        best_alpha = alpha
        best_score = score
print('best alpha = ', best_alpha)
print('best score = ', best_score)

best alpha =  0.001
best score =  24.085409252669038


In [162]:
model = smf.glm(formula=formula, data=sj, 
               family=sm.families.NegativeBinomial(alpha=best_alpha))

results = model.fit()
sj_pred = results.predict(sj_test).astype(int)
#confirm no negatives
np.min(sj_pred)

4

#### IQ

In [164]:
#create statsmodels formula
formula = 'total_cases ~ '
for i in iq.columns:
    if i != 'total_cases':
        formula = formula + str(i) + ' + '
formula =  formula[:-3] #trim the last plus sign
formula

'total_cases ~ precipitation_amt_mm + reanalysis_dew_point_temp_k + reanalysis_min_air_temp_k + reanalysis_specific_humidity_g_per_kg + reanalysis_tdtr_k + station_avg_temp_c + station_min_temp_c + fall + spring + summer + station_avg_temp_c_1lag + station_avg_temp_c_2lag + station_avg_temp_c_3lag + reanalysis_min_air_temp_k_1lag + reanalysis_min_air_temp_k_2lag + reanalysis_min_air_temp_k_3lag + station_min_temp_c_1lag + station_min_temp_c_2lag + station_min_temp_c_3lag + reanalysis_dew_point_temp_k_1lag + reanalysis_dew_point_temp_k_2lag + reanalysis_dew_point_temp_k_3lag + reanalysis_tdtr_k_1lag + reanalysis_tdtr_k_2lag + reanalysis_tdtr_k_3lag + reanalysis_specific_humidity_g_per_kg_1lag + reanalysis_specific_humidity_g_per_kg_2lag + reanalysis_specific_humidity_g_per_kg_3lag + precipitation_amt_mm_1lag + precipitation_amt_mm_2lag + precipitation_amt_mm_3lag'

In [165]:
iqtrain, iqtest, _, _ = train_test_split(
    iq, [0 for _ in range(iq.shape[0])], test_size=0.3)
for alpha in grid:
    model = smf.glm(formula=formula, data=iqtrain,
                    family=sm.families.NegativeBinomial(alpha=alpha))
    results = model.fit()
    predictions = results.predict(iqtest).astype(int)
    score = eval_measures.meanabs(predictions, iqtest.total_cases)
    best_score = 1000
    if score < best_score:
        best_alpha = alpha
        best_score = score
print('best alpha = ', best_alpha)
print('best score = ', best_score)

best alpha =  0.001
best score =  6.538461538461538


In [167]:
model = smf.glm(formula=formula, data=iq, 
               family=sm.families.NegativeBinomial(alpha=best_alpha))

results = model.fit()
iq_pred = results.predict(iq_test).astype(int)
#confirm no negatives
np.min(iq_pred)

1

In [169]:
submission = pd.read_csv('submission_format.csv',
                            index_col=[0, 1, 2])

In [170]:
submission.total_cases = np.concatenate([sj_pred, iq_pred])
submission.to_csv("NBR_with_3wk_lag.csv")